In [1]:
%time from pdb_profiling import default_config
%time from pdb_profiling.processors import Identifier, UniProtFASTA
%time from pdb_profiling.utils import a_seq_reader

%time default_config('C:/GitWorks/pdb-profiling/test/demo')

Wall time: 4.99 ms
Wall time: 2.47 s
Wall time: 0 ns
Wall time: 82.1 ms


In [2]:
demo = Identifier('NP_001291289.1')
demo

<RefSeq protein NP_001291289 1>

In [3]:
entry, isoform = demo.map2unp().result()
entry, isoform

('Q9C0B2', None)

In [4]:
demo.get_all_level_identifiers().result()

{'protein': 'NP_001291289.1', 'transcript': 'NM_001304360.1', 'gene': None}

In [5]:
'''
Identifier(
    demo.get_all_level_identifiers().result()['protein']
).fetch_sequence().result()
'''
np_header, np_seq = demo.fetch_sequence().result()
np_header, np_seq

('>NP_001291289.1 cilia- and flagella-associated protein 74 [Homo sapiens]',
 'MEDDGSLLPEDELLADALLLEDERDELEDPEFDIKCLLQEAEDDVDPGHSSSVKELDTDADKLKKKTAEDRTQAFHLRQNLSALDKMHEEQELFTEKMRGELRACRQRRDLIDKQQEAVAAEIATEEEAGNMAAVGRLQAVSRRLFAELENERDLQSRTEAVLKESENTMWHIEIQEGRLEAFRTADREEVEATGRRLQVRAAEQLCREQEALGKVERNRLLRIRKSLNTQKELGLRHQKLLEDARKNHKVAVRFLKASLGRIREQEKKEEMECHEYMRRRMDAVVALKGSISANRDTLRKFQAWDRAKAELAEQRVQAEKKAILAQGRDAFRHLVHQRRRQELEAQKRAFEEEQKLRKQEIISRILKEEAEEEKRKKQHPPTSARHRLTLRDKTWNYISDFCKKTTVPTNTYTLDYEAAAGPGPSRLLEVVSSELIQGDPGASSEEETLAEPEISGLWNEDYKPYQVPKEDVDRKPVGGTKMDKDILERTVERLRSRVVHKQVVWGREFQGRPFNSKPELLHFQDFDIGKVYKKKITLVNTTYTINYCKLVGVEEHLRDFIHVDFDPPGPLSAGMSCEVLVTFKPMINKDLEGNISFLAQTGEFSVPLKCSTKKCSLSLDKELIDFGSYVVGETTSRTITLTNVGGLGTTFKFLPASEPCEMDDSQSALKLSSLLTYEDKSLYDKAATSFSEQQLEGTESSQADMQSRKELEKLDKEQEEEQPAEPERLTTVIPPSEEQTEITLGEVTEGEIGPFSSIKVPIVFTPVVPGDVQARFKVTFKNPQCPTLHFRVVGVAIDVPVWVPKPSVDLKICMYDRLYQDSVLVHTRSKAALRLKFEVCKELRAHLELLPKTGYIQAQSSYSVQLKFLPRHSLPEDAGRYFDKETRVLEAPMTIWVADQNKPVGFTVHAIVTTSDLELS

In [6]:
demo.status

True

In [7]:
unp_header, unp_seq = UniProtFASTA.single_retrieve(entry, UniProtFASTA.folder, UniProtFASTA.web_semaphore).then(a_seq_reader).result()
unp_header, unp_seq

('>sp|Q9C0B2|CFA74_HUMAN Cilia- and flagella-associated protein 74 OS=Homo sapiens OX=9606 GN=CFAP74 PE=2 SV=3',
 'MEDDGSLLPEDELLADALLLEDERDELEDPEFDIKCLLQEAEDDVDPGHSSSVKELDTDADKLKKKTAEDRTQAFHLRQNLSALDKMHEEQELFTEKMRGELRACRQRRDLIDKQQEAVAAEIATEEEAGNMAAVGRLQAVSRRLFAELENERDLQSRTEAVLKESENTMWHIEIQEGRLEAFRTADREEVEATGRRLQVRAAEQLCREQEALGKVERNRLLRIRKSLNTQKELGLRHQKLLEDARKNHKVAVRFLKASLGRIREQEKKEEMECHEYMRRRMDAVVALKGSISANRDTLRKFQAWDRAKAELAEQRVQAEKKAILAQGRDAFRHLVHQRRRQELEAQKRAFEEEQKLRKQEIISRILKEEAEEEKRKKQHPPTSARHRLTLRDKTWNYISDFCKKTTVPTNTYTLDYEAAAGPGPSRLLEVVSSELIQGDPGASSEEETLAEPEISGLWNEDYKPYQVPKEDVDRKPVGGTKMDKDILERTVERLRSRVVHKQVVWGREFQGRPFNSKPELLHFQDFDIGKVYKKKITLVNTTYTINYCKLVGVEEHLRDFIHVDFDPPGPLSAGMSCEVLVTFKPMINKDLEGNISFLAQTGEFSVPLKCSTKKCSLSLDKELIDFGSYVVGETTSRTITLTNVGGLGTTFKFLPASEPCEMDDSQSALKLSSLLTYEDKSLYDKAATSFSEQQLEGTESSQADMQSRKELEKLDKEQEEEQPAEPERLTTVIPPSEEQTEITLGEVTEGEIGPFSSIKVPIVFTPVVPGDVQARFKVTFKNPQCPTLHFRVVGVAIDVPVWVPKPSVDLKICMYDRLYQDSVLVHTRSKAALRLKFEVCKELRAHLELLPKTGYIQAQSSYSVQLKFLPRHSLPEDAGRYFDK

```py
from dtaidistance import alignment
%time value, matrix = alignment.needleman_wunsch(unp_seq, np_seq)
algn, s1a, s2a = alignment.best_alignment(matrix, unp_seq, np_seq, gap='-')
print(''.join(s1a[1500:]))
print(''.join(s2a[1500:]))

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.figure(figsize=(10,8))
sns.heatmap(matrix[1500:,1500:],cmap='icefire')
plt.show()
```

<img src="../docs/figs/ToUniProt_ali.svg">

In [8]:
from Bio import Align
%time aligner = Align.PairwiseAligner()
%time alignments = aligner.align(unp_seq, np_seq)
%time alignments[0].aligned

Wall time: 0 ns
Wall time: 20.9 ms
Wall time: 0 ns


(((0, 1507), (1507, 1551), (1551, 1552), (1552, 1553), (1553, 1584)),
 ((0, 1507), (1517, 1561), (1581, 1582), (1603, 1604), (1608, 1639)))

In [9]:
for i in str(alignments[0]).split('\n'):
    print(i[1500:])

FDPRHRE----------EAEELRPILVTLDYIQFDTDTPAPPATRELQVGCIRTTQPSPKK--------------------P---------------------D----HPLMVSALLQLRGDVKETYKVIFVAQVLTGP
|||||||----------||||||||||||||||||||||||||||||||||||||||||||--------------------|---------------------|----|||||||||||||||||||||||||||||||
FDPRHREASSRPGPLSPEAEELRPILVTLDYIQFDTDTPAPPATRELQVGCIRTTQPSPKKTVEFSIDSVASLQHKGFSIEPSRGSVERGQTKTISISWVPPADFDPDHPLMVSALLQLRGDVKETYKVIFVAQVLTGP

